In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

In [2]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.charts import Bar
from bokeh.io import output_notebook
from bokeh.palettes import Spectral6
from IPython.html.widgets import *

C:\Users\Ofer\Anaconda2\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
output_notebook()
TOOLS = "pan, box_zoom, wheel_zoom, reset, save, hover"

Loading BokehJS ...

In [ ]:
labor_df = pd.read_excel('./Data/oesm15all/all_data_m_2015.xlsx')

In [4]:
occ_head_dict = {
11:"Management Occupations",13: "Business and Financial Operations Occupations",15: "Computer and Mathematical Occupations",
17: "Architecture and Engineering Occupations", 19: "Life, Physical, and Social Science Occupations", 
21: "Community and Social Service Occupations", 23: "Legal Occupations", 25: "Education, Training, and Library Occupations",
27: "Arts, Design, Entertainment, Sports, and Media Occupations", 29: "Healthcare Practitioners and Technical Occupations",
31: "Healthcare Support Occupations", 33: "Protective Service Occupations", 35: "Food Preparation and Serving Related Occupations",
37: "Building and Grounds Cleaning and Maintenance Occupations", 39: "Personal Care and Service Occupations",
41: "Sales and Related Occupations", 43: "Office and Administrative Support Occupations", 45: "Farming, Fishing, and Forestry Occupations",
47: "Construction and Extraction Occupations", 49: "Installation, Maintenance, and Repair Occupations", 51: "Production Occupations",
53: "Transportation and Material Moving Occupations", 55: "Military Occupations",
99: "Unemployed and last worked 5 years ago or earlier or never worked"
}

race_dict = {
1: "White alone", 2: "Black or African American alone", 3: "American Indian alone", 4: "Alaska Native alone", 
5: "American Indian or Alaska Native", 6: "Asian alone", 7: "Native Hawaiian and Other Pacific Islander alone",
8: "Some Other Race alone", 9: "Two or More Races"
}

state_dict = {
1: "AL", 2: "AK", 4: "AR", 5: "AR", 6: "CA", 8: "CO", 9: "CT", 10: "DE", 11: "DC", 12: "FL", 13: "GA",
15: "HI", 16: "ID", 17: "IL", 18: "IN", 19: "IA", 20: "KS", 21: "KY", 22: "LA", 23: "ME", 24: "MD", 25: "MA",
26: "MI", 27: "MN", 28: "MS", 29: "MO", 30: "MT", 31: "NE", 32: "NV", 33: "NH", 34: "NJ", 35: "NM", 36: "NY", 
37: "NC", 38: "ND", 39: "OH", 40: "OK", 41: "OR", 42: "PA", 44: "RI", 45: "SC", 46: "SD", 47: "TN", 48: "TX",
49: "UT", 50: "VT", 51: "VA", 53: "WA", 54: "WV", 55: "WI", 56: "WY", 72: "PR"      
}

In [5]:
columns = ['SERIALNO','SPORDER','ST','SEX','AGEP','RAC1P','PERNP', 'PINCP','WAGP','SCHL','SOCP']
names = ['house_number','person_number','state_code','sex','age','race_code','earnings','income','wages','ed_level','occupation_code']
df_raw_a = pd.read_csv('./Data/pums_2014_1yr/ss14pusa.csv',usecols=columns)\
          .rename(columns={raw:name for raw, name in zip(columns, names)})
df_raw_b = pd.read_csv('./Data/pums_2014_1yr/ss14pusb.csv',usecols=columns)\
          .rename(columns={raw:name for raw, name in zip(columns, names)})
df = pd.concat([df_raw_a, df_raw_b], axis=0)

# Create occupation buckets
df['occupation'] = pd.Series([occ_head_dict[int(occ[:2])] if pd.isnull(occ)==False else 'None' for occ in df.occupation_code])

# Create race descriptions
df['race'] = pd.Series([race_dict[int(race)] if pd.isnull(race)==False else 'None' for race in df.race_code])

# Adjust wage information
adj_fact = 1.008425 ## Adjustment factor to 2014 dollar amounts (from data)
df.wages.loc[df.wages!= ' '] = df.wages.copy().loc[df.wages!= ' '].map(float)*adj_fact

# Adjust sex information
df.sex.loc[df.sex==1]='Male'
df.sex.loc[df.sex==2]='Female'

# Enter state name
df['state'] = pd.Series([state_dict[state] for state in df.state_code])


C:\Users\Ofer\Anaconda2\lib\site-packages\pandas\core\indexing.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
stat= 'VA'
occ = 'yo mama'
p = Bar(plot_df[:100], label='race', values='wages', agg='median', group='sex',
        title='Median wage in\n{0} by race, \n grouped by gender, in {1}'.format(occ,stat),
        legend='top_right', plot_width=700)

show(p)

In [16]:
# Setting up the dataframe
plot_df = df[['state','sex','race','wages', 'occupation']].copy()
plot_df = plot_df.loc[plot_df['wages']!=' ' ]
plot_df = plot_df.loc[plot_df['wages']!=0]

def update(lag=5, state='VA', occupation='Management Occupations'):
    #colorwheel = Spectral16
    #color_idx = 0
    
    # Filtering the data 
    #males = df[(df[Gender]=='Male') & (df[Year]==2014)]
    p_df = plot_df.loc[(plot_df['state']==state)&(plot_df['occupation']==occupation)]
    #p_df = p_df.loc[]

    
    p = Bar(plot_df, label='race', values='wages', agg='median', group='sex',
            title="Median wage in {0} by race, grouped by gender in {1}".format(occupation, state),
            legend='top_right', tools=TOOLS, plot_width=700)
    
    show(p)

In [17]:
interact(update, occupation=('Management Occupations', 'Business and Financial Operations Occupations',
                             'Life, Physical, and Social Science Occupations',
                             'Architecture and Engineering Occupations',
                             'Computer and Mathematical Occupations',
                             'Healthcare Practitioners and Technical Occupations',
                             'Healthcare Support Occupations'),state=('VA','NY','DC'), lag=(5,360,5))